In [37]:
import pandas as pd
import numpy as np
from datetime import datetime
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from slugify import slugify

In [38]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [39]:
df = pd.read_csv('csv/localities.csv', encoding='utf-8')
image = pd.read_csv('out/image.csv', encoding='utf-8')

In [40]:
newDfC = pd.DataFrame(data = df['Continent'].unique())

In [41]:
columns = ['id', 'createdDate', 'updatedDate', 'latitude', 'longitude', 'internalNote', \
          'isPopular', 'order', 'statusId', 'canShowAds', 'isActive', 'slug', \
          'mainImageId', 'coverImageId', 'type', 'countryContinentId', 'regionCountryId', \
          'regionContinentId']

In [42]:
dfCon = newDfC

In [44]:
dfCon = pd.concat([dfCon, pd.DataFrame(columns = columns)])

In [45]:
dfCon['id'] = dfCon.index + 1
dfCon['order'] = ''
dfCon['createdDate'] = dt_string
dfCon['updatedDate'] = dt_string
dfCon['internalNote'] = ''
dfCon['isPopular'] = False
dfCon['statusId'] = 7
dfCon['canShowAds'] = False
dfCon['isActive'] = True
dfCon['type'] = 'Continent'

In [46]:
dfCon.rename(columns = {0:'name'}, inplace = True)

In [47]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="trynego")

In [48]:
def get_lat(value):
    try:
        return geolocator.geocode(str(value))[1][0]
    except:
        return ''

In [49]:
def get_lot(value):
    try:
        return geolocator.geocode(str(value))[1][1]
    except:
        return ''

In [50]:
def set_slug(value):
    return slugify(str(value))

In [51]:
dfCon['slug'] = dfCon['name'].apply(set_slug)

In [288]:
dfCon['latitude'] = dfCon['name'].apply(get_lat)
dfCon['longitude'] = dfCon['name'].apply(get_lot)

In [53]:
newDfCountries = pd.DataFrame(data = df['Country'].unique())

In [54]:
dfCountry = pd.concat([newDfCountries, pd.DataFrame(columns = columns)])

In [55]:
dfCountry['id'] = dfCountry.index + 6
dfCountry['order'] = ''
dfCountry['createdDate'] = dt_string
dfCountry['updatedDate'] = dt_string
dfCountry['internalNote'] = ''
dfCountry['isPopular'] = False
dfCountry['statusId'] = 7
dfCountry['canShowAds'] = False
dfCountry['isActive'] = True
dfCountry['type'] = 'Country'

In [56]:
dfCountry.rename(columns = {0:'name'}, inplace = True)
dfCountry['slug'] = dfCountry['name'].apply(set_slug)

In [ ]:
dfCountry['latitude'] = dfCountry['name'].apply(get_lat)
dfCountry['longitude'] = dfCountry['name'].apply(get_lot)

In [57]:
def set_continent_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Country'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [58]:
def set_image_id_c(value):
    try:
        return image[:][image['name'] == str(value).lower()]['id'].values[0]
    except:
        return ''

In [59]:
dfCountry['countryContinentId'] = dfCountry['name'].apply(set_continent_id)
dfCountry['coverImageId'] = dfCountry['name'].apply(set_image_id_c)

In [60]:
newDfR = pd.DataFrame(data = df['Region'].unique())

In [61]:
dfReg = pd.concat([newDfR, pd.DataFrame(columns = columns)])

In [62]:
dfReg['id'] = dfReg.index + 50
dfReg['order'] = ''
dfReg['createdDate'] = dt_string
dfReg['updatedDate'] = dt_string
dfReg['internalNote'] = ''
dfReg['isPopular'] = False
dfReg['statusId'] = 7
dfReg['canShowAds'] = False
dfReg['isActive'] = True
dfReg['type'] = 'Region'

In [63]:
dfReg.rename(columns = {0:'name'}, inplace = True)
dfReg['slug'] = dfReg['name'].apply(set_slug)

In [ ]:
dfReg['latitude'] = dfReg['name'].apply(get_lat)
dfReg['longitude'] = dfReg['name'].apply(get_lot)

In [64]:
def set_continent_reg_id(value):
    try:
        return dfCon[:][dfCon['name'] == df[:][df['Region'] == str(value)]['Continent'].values[0]]['id'].values[0]
    except:
        return ''

In [65]:
def set_region_reg_id(value):
    try:
        return dfCountry[:][dfCountry['name'] == df[:][df['Region'] == str(value)]['Country'].values[0]]['id'].values[0]
    except:
        return ''

In [66]:
dfReg['regionContinentId'] = dfReg['name'].apply(set_continent_reg_id)

In [67]:
dfReg['regionCountryId'] = dfReg['name'].apply(set_region_reg_id)

In [85]:
def set_image_id(value):
    try:
        return image[:][image['name'] == str(value).rpartition(' ')[-1].lower()]['id'].values[0]
    except:
        print(str(value).rpartition(' ')[-1].lower())
        return ''

In [95]:
dfReg['coverImageId'] = dfReg['name'].apply(set_image_id)

-
baños
galápagos
lópez
medellín
florianópolis
santarém
brasília
río
cancún
potosí
nan


In [97]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [98]:
newDf = pd.concat([dfCon, dfCountry, dfReg])

In [106]:
newDf = newDf[:][(newDf['name'] != '') & (newDf['name'] != '-') & (newDf['name'].notnull())]

In [107]:
newDf['id'] = [i for i in range(1, newDf.shape[0] + 1)]

In [100]:
dfCountry['order'] = dfCountry.index + 1

In [101]:
dfCountry.to_csv('out/countries.csv', index=False, encoding='utf-8')

In [108]:
newDf.to_csv('out/localities.csv', index=False, encoding='utf-8')